# Tools for Crystal Symmetry Analysis

Crystallographic symmetry analysis which can be necessary to interpret EBSD data can be carried out using the [pymatgen](http://pymatgen.org/) ("Python Materials Genomics") library [1].

The crystallographic analysis of the structure data is provided in pymatgen via the powerful [spglib](https://atztogo.github.io/spglib/python-spglib.html#python-spglib) library, which also defines the [crystallographic conventions](https://atztogo.github.io/spglib/definition.html) used in space group analysis. 

Pymatgen includes routines to access crystal structure data in various input formats, including data import directly from the online [Crystallography Open Database](http://www.crystallography.net/cod/).

[1]
Shyue Ping Ong, William Davidson Richards, Anubhav Jain, Geoffroy Hautier, Michael Kocher, Shreyas Cholia, Dan Gunter, Vincent Chevrier, Kristin A. Persson, Gerbrand Ceder. _Python Materials Genomics (pymatgen) : A Robust, Open-Source Python Library for Materials Analysis._ Computational Materials Science, 2013, 68, 314–319. [doi:10.1016/j.commatsci.2012.10.028](http://dx.doi.org/10.1016/j.commatsci.2012.10.028) 


In [1]:
import numpy as np
import pymatgen as mg
from pymatgen.io.cif import CifParser
from pymatgen.ext.cod import COD
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

## Importing Structure Data

In [2]:
#this gets the primitive cell
parser = CifParser("Cu-Copper.cif")
prim_structure = parser.get_structures()[0]
print(prim_structure)

Full Formula (Cu1)
Reduced Formula: Cu
abc   :   2.556163   2.556163   2.556163
angles:  60.000000  60.000000  60.000000
Sites (1)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Cu      0    0    0


In [3]:
cod = COD()
cod_structure = cod.get_structure_by_id(1010064)
print(cod_structure)

Full Formula (Li8 O4)
Reduced Formula: Li2O
abc   :   4.610000   4.610000   4.610000
angles:  90.000000  90.000000  90.000000
Sites (12)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Li+   0.25  0.25  0.25
  1  Li+   0.25  0.75  0.75
  2  Li+   0.75  0.25  0.75
  3  Li+   0.75  0.75  0.25
  4  Li+   0.75  0.75  0.75
  5  Li+   0.75  0.25  0.25
  6  Li+   0.25  0.75  0.25
  7  Li+   0.25  0.25  0.75
  8  O2-   0     0     0
  9  O2-   0     0.5   0.5
 10  O2-   0.5   0     0.5
 11  O2-   0.5   0.5   0


In [4]:
# Reading a structure from CIF
#structure = mg.Structure.from_str(open("CsCl.cif").read(), fmt="cif")
structure = mg.Structure.from_file("Cu-Copper.cif")
print(structure)

Full Formula (Cu4)
Reduced Formula: Cu
abc   :   3.614960   3.614960   3.614960
angles:  90.000000  90.000000  90.000000
Sites (4)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Cu    0    0    0
  1  Cu    0    0.5  0.5
  2  Cu    0.5  0    0.5
  3  Cu    0.5  0.5  0


## Space Group Assignment 

In [5]:
finder = SpacegroupAnalyzer(structure)
print("The space group symbol is {}".format(finder.get_space_group_symbol()))
print("The point group symbol is {}".format(finder.get_point_group_symbol()))

#structure=finder.get_symmetrized_structure()
#structure =finder.get_conventional_standard_structure()
#print(structure)

The space group symbol is Fm-3m
The point group symbol is m-3m


We can output a dictionary of all the symmetry data that has been determined by SpacegroupAnalyzer via spglib: 

In [6]:
symdata=finder.get_symmetry_dataset()
print(symdata)

{'number': 225, 'hall_number': 523, 'international': 'Fm-3m', 'hall': '-F 4 2 3', 'choice': '', 'transformation_matrix': array([[  1.00000000e+00,   3.79646512e-17,   0.00000000e+00],
       [  1.91466924e-19,   1.91466924e-19,  -1.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00]]), 'origin_shift': array([ 1.,  1.,  1.]), 'rotations': array([[[ 1,  0,  0],
        [ 0,  1,  0],
        [ 0,  0,  1]],

       [[-1,  0,  0],
        [ 0, -1,  0],
        [ 0,  0, -1]],

       [[ 0,  0,  1],
        [ 0,  1,  0],
        [-1,  0,  0]],

       ..., 
       [[ 0,  0,  1],
        [ 1,  0,  0],
        [ 0, -1,  0]],

       [[ 1,  0,  0],
        [ 0,  0, -1],
        [ 0,  1,  0]],

       [[-1,  0,  0],
        [ 0,  0,  1],
        [ 0, -1,  0]]], dtype=int32), 'translations': array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  5.00000000e-01,   1.00000000e+00,   5.00000000e-01],
       [  1.00000000e+00,   0.00000000e+00,   0.00000000e+

Getting the unique point group operators:

In [7]:
pg = np.array([ [[ 1.,  0.,  0.],
                 [ 0.,  1.,  0.],
                 [ 0.,  0.,  1.]] ] )
for idx,op in enumerate(symdata['rotations']):
    is_new_op=True
    for pg_op in pg:
        if np.array_equal(pg_op,op):
            is_new_op=False
    if is_new_op:         
        pg=np.append(pg,[op],axis=0)
#print(pg)   
print(pg.shape)

(48, 3, 3)


In [8]:
hkl=np.array([1,0,0])
new_hkl=np.matmul(hkl,pg)
print(new_hkl.shape)
print(new_hkl)

(48, 3)
[[ 1.  0.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0. -1.]
 [-1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0. -1.]
 [-1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0. -1.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  0.]
 [ 0. -1.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0. -1.]
 [-1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0. -1.]
 [-1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [-1.  0.  0.]]


In [9]:
# sort out the unique hkls of the family 
hkl_family=np.array([new_hkl[0]])
for idx,hkl in enumerate(new_hkl):
    is_new_hkl=True
    for f in hkl_family:
        if np.array_equal(hkl,f):
            is_new_hkl=False
    if is_new_hkl:         
        hkl_family=np.append(hkl_family,[hkl],axis=0)
        
print(hkl_family.shape)
print(hkl_family)   

(6, 3)
[[ 1.  0.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0. -1.]
 [ 0.  1.  0.]
 [ 0. -1.  0.]]


In [10]:
# alternative approach

# returns a list of SymmOp objects
pg_ops=finder.get_point_group_operations()

hkl=np.array([0,0,1])
family=np.array([hkl])

for idx,op in enumerate(pg_ops):
    new_hkl=pg_ops[idx].apply_rotation_only(hkl)
    hkl_is_new=True
    for fvec in family:
        if np.array_equal(new_hkl,fvec):
            hkl_is_new=False
    if hkl_is_new:
        family=np.append(family,[new_hkl],axis=0)
print(hkl, " family:")
print(family)

[0 0 1]  family:
[[ 0.  0.  1.]
 [ 0.  0. -1.]
 [ 1.  0.  0.]
 [-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  1.  0.]]


## spglib Examples

In [11]:
import sys
import spglib
import numpy as np

#######################################
# Uncomment to use Atoms class in ASE #
#######################################
# from ase import Atoms

#######################################################################
# Using the local Atoms-like class (BSD license) where a small set of #
# ASE Atoms features is comaptible but enough for this example.       #
#######################################################################
from atoms import Atoms

In [12]:
def show_symmetry(symmetry):
    for i in range(symmetry['rotations'].shape[0]):
        print("  --------------- %4d ---------------" % (i + 1))
        rot = symmetry['rotations'][i]
        trans = symmetry['translations'][i]
        print("  rotation:")
        for x in rot:
            print("     [%2d %2d %2d]" % (x[0], x[1], x[2]))
        print("  translation:")
        print("     (%8.5f %8.5f %8.5f)" % (trans[0], trans[1], trans[2]))

def show_lattice(lattice):
    print("Basis vectors:")
    for vec, axis in zip(lattice, ("a", "b", "c")):
        print("%s %10.5f %10.5f %10.5f" % (tuple(axis,) + tuple(vec)))

def show_cell(lattice, positions, numbers):
    show_lattice(lattice)
    print("Atomic points:")
    for p, s in zip(positions, numbers):
        print("%2d %10.5f %10.5f %10.5f" % ((s,) + tuple(p)))

In [13]:
silicon_ase = Atoms(symbols=['Si'] * 8,
                    cell=[(4, 0, 0),
                          (0, 4, 0),
                          (0, 0, 4)],
                    scaled_positions=[(0, 0, 0),
                                      (0, 0.5, 0.5),
                                      (0.5, 0, 0.5),
                                      (0.5, 0.5, 0),
                                      (0.25, 0.25, 0.25),
                                      (0.25, 0.75, 0.75),
                                      (0.75, 0.25, 0.75),
                                      (0.75, 0.75, 0.25)],
                    pbc=True)

silicon = ([(4, 0, 0),
            (0, 4, 0),
            (0, 0, 4)],
           [(0, 0, 0),
            (0, 0.5, 0.5),
            (0.5, 0, 0.5),
            (0.5, 0.5, 0),
            (0.25, 0.25, 0.25),
            (0.25, 0.75, 0.75),
            (0.75, 0.25, 0.75),
            (0.75, 0.75, 0.25)],
           [14,] * 8)

silicon_dist = ([(4.01, 0, 0),
                 (0, 4, 0),
                 (0, 0, 3.99)],
                [(0.001, 0, 0),
                 (0, 0.5, 0.5),
                 (0.5, 0, 0.5),
                 (0.5, 0.5, 0),
                 (0.25, 0.25, 0.251),
                 (0.25, 0.75, 0.75),
                 (0.75, 0.25, 0.75),
                 (0.75, 0.75, 0.25)],
                [14,] * 8)

silicon_prim = ([(0, 2, 2),
                 (2, 0, 2),
                 (2, 2, 0)],
                [(0, 0, 0),
                 (0.25, 0.25, 0.25)],
                [14, 14])

rutile = ([(4, 0, 0),
           (0, 4, 0),
           (0, 0, 3)],
          [(0, 0, 0),
           (0.5, 0.5, 0.5),
           (0.3, 0.3, 0.0),
           (0.7, 0.7, 0.0),
           (0.2, 0.8, 0.5),
           (0.8, 0.2, 0.5)],
          [14, 14, 8, 8, 8, 8])

rutile_dist = ([(3.97, 0, 0),
                (0, 4.03, 0),
                (0, 0, 3.0)],
               [(0, 0, 0),
                (0.5001, 0.5, 0.5),
                (0.3, 0.3, 0.0),
                (0.7, 0.7, 0.002),
                (0.2, 0.8, 0.5),
                (0.8, 0.2, 0.5)],
               [14, 14, 8, 8, 8, 8])

a = 3.07
c = 3.52
MgB2 = ([(a, 0, 0),
         (-a/2, a/2*np.sqrt(3), 0),
         (0, 0, c)],
        [(0, 0, 0),
         (1.0/3, 2.0/3, 0.5),
         (2.0/3, 1.0/3, 0.5)],
        [12, 5, 5])

a = [3., 0., 0.]
b = [-3.66666667, 3.68178701, 0.]
c = [-0.66666667, -1.3429469, 1.32364995]
niggli_lattice = np.array([a, b, c])

In [14]:
print("[get_spacegroup]")
print("  Spacegroup of Silicon is %s." % spglib.get_spacegroup(silicon))
print('')

print("[get_spacegroup]")
print("  Spacegroup of Silicon (ASE Atoms-like format) is %s." %
      spglib.get_spacegroup(silicon_ase))
print('')
print("[get_spacegroup]")
print("  Spacegroup of Rutile is %s." % spglib.get_spacegroup(rutile))
print('')
print("[get_spacegroup]")
print("  Spacegroup of MgB2 is %s." % spglib.get_spacegroup(MgB2))
print('')
print("[get_symmetry]")
print("  Symmetry operations of Rutile unitcell are:")
print('')
symmetry = spglib.get_symmetry(rutile)
show_symmetry(symmetry)
print('')
print("[get_symmetry]")
print("  Symmetry operations of MgB2 are:")
print('')
symmetry = spglib.get_symmetry(MgB2)
show_symmetry(symmetry)
print('')
print("[get_pointgroup]")
print("  Pointgroup of Rutile is %s." %
      spglib.get_pointgroup(symmetry['rotations'])[0])
print('')

dataset = spglib.get_symmetry_dataset( rutile )
print("[get_symmetry_dataset] ['international']")
print("  Spacegroup of Rutile is %s (%d)." % (dataset['international'],
                                              dataset['number']))
print('')
print("[get_symmetry_dataset] ['pointgroup']")
print("  Pointgroup of Rutile is %s." % (dataset['pointgroup']))
print('')
print("[get_symmetry_dataset] ['hall']")
print("  Hall symbol of Rutile is %s (%d)." % (dataset['hall'],
                                               dataset['hall_number']))
print('')
print("[get_symmetry_dataset] ['wyckoffs']")
alphabet = "abcdefghijklmnopqrstuvwxyz"
print("  Wyckoff letters of Rutile are: ", dataset['wyckoffs'])
print('')
print("[get_symmetry_dataset] ['equivalent_atoms']")
print("  Mapping to equivalent atoms of Rutile are: ")
for i, x in enumerate(dataset['equivalent_atoms']):
  print("  %d -> %d" % (i + 1, x + 1))
print('')
print("[get_symmetry_dataset] ['rotations'], ['translations']")
print("  Symmetry operations of Rutile unitcell are:")
for i, (rot,trans) in enumerate(zip(dataset['rotations'],
                                    dataset['translations'])):
    print("  --------------- %4d ---------------" % (i + 1))
    print("  rotation:")
    for x in rot:
        print("     [%2d %2d %2d]" % (x[0], x[1], x[2]))
    print("  translation:")
    print("     (%8.5f %8.5f %8.5f)" % (trans[0], trans[1], trans[2]))
print('')

print("[refine_cell]")
print(" Refine distorted rutile structure")
lattice, positions, numbers = spglib.refine_cell(rutile_dist, symprec=1e-1)
show_cell(lattice, positions, numbers)
print('')

print("[find_primitive]")
print(" Fine primitive distorted silicon structure")
lattice, positions, numbers = spglib.find_primitive(silicon_dist, symprec=1e-1)
show_cell(lattice, positions, numbers)
print('')

print("[standardize_cell]")
print(" Standardize distorted rutile structure:")
print(" (to_primitive=0 and no_idealize=0)")
lattice, positions, numbers = spglib.standardize_cell(rutile_dist,
                                                      to_primitive=0,
                                                      no_idealize=0,
                                                      symprec=1e-1)
show_cell(lattice, positions, numbers)
print('')

print("[standardize_cell]")
print(" Standardize distorted rutile structure:")
print(" (to_primitive=0 and no_idealize=1)")
lattice, positions, numbers = spglib.standardize_cell(rutile_dist,
                                                      to_primitive=0,
                                                      no_idealize=1,
                                                      symprec=1e-1)
show_cell(lattice, positions, numbers)
print('')

print("[standardize_cell]")
print(" Standardize distorted silicon structure:")
print(" (to_primitive=1 and no_idealize=0)")
lattice, positions, numbers = spglib.standardize_cell(silicon_dist,
                                                      to_primitive=1,
                                                      no_idealize=0,
                                                      symprec=1e-1)
show_cell(lattice, positions, numbers)
print('')

print("[standardize_cell]")
print(" Standardize distorted silicon structure:")
print(" (to_primitive=1 and no_idealize=1)")
lattice, positions, numbers = spglib.standardize_cell(silicon_dist,
                                                      to_primitive=1,
                                                      no_idealize=1,
                                                      symprec=1e-1)
show_cell(lattice, positions, numbers)
print('')

symmetry = spglib.get_symmetry(silicon)
print("[get_symmetry]")
print("  Number of symmetry operations of silicon conventional")
print("  unit cell is %d (192)." % len(symmetry['rotations']))
show_symmetry(symmetry)
print('')

symmetry = spglib.get_symmetry_from_database(525)
print("[get_symmetry_from_database]")
print("  Number of symmetry operations of silicon conventional")
print("  unit cell is %d (192)." % len(symmetry['rotations']))
show_symmetry(symmetry)
print('')

reduced_lattice = spglib.niggli_reduce(niggli_lattice)
print("[niggli_reduce]")
print("  Original lattice")
show_lattice(niggli_lattice)
print("  Reduced lattice")
show_lattice(reduced_lattice)
print('')


mapping, grid = spglib.get_ir_reciprocal_mesh([11, 11, 11],
                                              silicon_prim,
                                              is_shift=[0, 0, 0])
num_ir_kpt = len(np.unique(mapping))
print("[get_ir_reciprocal_mesh]")
print("  Number of irreducible k-points of primitive silicon with")
print("  11x11x11 Monkhorst-Pack mesh is %d (56)." % num_ir_kpt)
print('')

mapping, grid = spglib.get_ir_reciprocal_mesh([8, 8, 8],
                                              rutile,
                                              is_shift=[1, 1, 1])
num_ir_kpt = len(np.unique(mapping))
print("[get_ir_reciprocal_mesh]")
print("  Number of irreducible k-points of Rutile with")
print("  8x8x8 Monkhorst-Pack mesh is %d (40)." % num_ir_kpt)
print('')

mapping, grid = spglib.get_ir_reciprocal_mesh([9, 9, 8],
                                              MgB2,
                                              is_shift=[0, 0, 1])
num_ir_kpt = len(np.unique(mapping))
print("[get_ir_reciprocal_mesh]")
print("  Number of irreducible k-points of MgB2 with")
print("  9x9x8 Monkhorst-Pack mesh is %s (48)." % num_ir_kpt)
print('')

[get_spacegroup]
  Spacegroup of Silicon is Fd-3m (227).

[get_spacegroup]
  Spacegroup of Silicon (ASE Atoms-like format) is Fd-3m (227).

[get_spacegroup]
  Spacegroup of Rutile is P4_2/mnm (136).

[get_spacegroup]
  Spacegroup of MgB2 is P6/mmm (191).

[get_symmetry]
  Symmetry operations of Rutile unitcell are:

  ---------------    1 ---------------
  rotation:
     [ 1  0  0]
     [ 0  1  0]
     [ 0  0  1]
  translation:
     ( 0.00000  0.00000  0.00000)
  ---------------    2 ---------------
  rotation:
     [-1  0  0]
     [ 0 -1  0]
     [ 0  0 -1]
  translation:
     ( 0.00000  0.00000  0.00000)
  ---------------    3 ---------------
  rotation:
     [ 0 -1  0]
     [ 1  0  0]
     [ 0  0  1]
  translation:
     ( 0.50000  0.50000  0.50000)
  ---------------    4 ---------------
  rotation:
     [ 0  1  0]
     [-1  0  0]
     [ 0  0 -1]
  translation:
     ( 0.50000  0.50000  0.50000)
  ---------------    5 ---------------
  rotation:
     [-1  0  0]
     [ 0 -1  0]
     [ 

  ---------------  130 ---------------
  rotation:
     [ 0 -1  0]
     [-1  0  0]
     [ 0  0  1]
  translation:
     ( 0.00000  0.50000  0.50000)
  ---------------  131 ---------------
  rotation:
     [ 0 -1  0]
     [ 0  0 -1]
     [-1  0  0]
  translation:
     ( 0.75000  0.75000  0.25000)
  ---------------  132 ---------------
  rotation:
     [ 0 -1  0]
     [ 1  0  0]
     [ 0  0 -1]
  translation:
     ( 0.00000  0.00000  0.00000)
  ---------------  133 ---------------
  rotation:
     [ 0  1  0]
     [ 0  0 -1]
     [ 1  0  0]
  translation:
     ( 0.75000  0.75000  0.25000)
  ---------------  134 ---------------
  rotation:
     [ 0  1  0]
     [ 1  0  0]
     [ 0  0  1]
  translation:
     ( 0.50000  0.00000  0.50000)
  ---------------  135 ---------------
  rotation:
     [ 0  1  0]
     [ 0  0  1]
     [-1  0  0]
  translation:
     ( 0.75000  0.75000  0.25000)
  ---------------  136 ---------------
  rotation:
     [ 0  1  0]
     [-1  0  0]
     [ 0  0 -1]
  translation

     [ 0  1  0]
     [ 1  0  0]
  translation:
     ( 0.75000  0.25000  0.75000)
  ---------------  161 ---------------
  rotation:
     [ 0  1  0]
     [ 0  0  1]
     [ 1  0  0]
  translation:
     ( 0.50000  0.50000  0.00000)
  ---------------  162 ---------------
  rotation:
     [ 1  0  0]
     [ 0  0  1]
     [ 0 -1  0]
  translation:
     ( 0.75000  0.75000  0.25000)
  ---------------  163 ---------------
  rotation:
     [ 0 -1  0]
     [ 0  0  1]
     [-1  0  0]
  translation:
     ( 0.00000  0.00000  0.00000)
  ---------------  164 ---------------
  rotation:
     [-1  0  0]
     [ 0  0  1]
     [ 0  1  0]
  translation:
     ( 0.75000  0.25000  0.75000)
  ---------------  165 ---------------
  rotation:
     [ 0 -1  0]
     [ 0  0 -1]
     [ 1  0  0]
  translation:
     ( 0.50000  0.50000  0.00000)
  ---------------  166 ---------------
  rotation:
     [-1  0  0]
     [ 0  0 -1]
     [ 0 -1  0]
  translation:
     ( 0.25000  0.25000  0.25000)
  ---------------  167 --------